In [19]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import operator
import os
import PIL
from PIL import Image
from tqdm import tqdm_notebook as tqdm

from shutil import copyfile
import shutil

import pickle

# keras : librairie de deep learning
import keras
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, InputLayer, ReLU, AveragePooling2D, Softmax
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16

from keras.models import load_model

In [20]:
project_path = './../' 
Airliners_path = project_path + 'Scraping/Airliners/data'
new_paths = [project_path + 'Split_Data/Airliners/Train', project_path + 'Split_Data/Airliners/Test']
airbus_planes = ['A320', 'A321', 'A330', 'A350']
boeing_planes =  ['737', '747', '757', '777']

# nb_types : number of classes to predict
nb_types = len(airbus_planes) + len(boeing_planes)

model_name = 'Ext_F_2'

In [21]:
%run g7_functions_for_models_V2.ipynb
#%run g7_data_augmentation.ipynb

# Lecure avec keras

In [22]:
sep_train_test_airliners(Airliners_path, new_paths, airbus_planes = airbus_planes, boeing_planes = boeing_planes)

In [23]:
def data_augmentation(train_path, coeff_creation = 2, rotation_range = 10, width_shift_range = .2, 
                      height_shift_range = .2, shear_range = .2, zoom_range = .2, horizontal_flip = True, 
                      nb_img = 10, save_format = 'jpg'):
   
    classes = os.listdir(train_path)
 
    datagen = ImageDataGenerator(
           rotation_range=rotation_range,
           width_shift_range=width_shift_range,
           height_shift_range=height_shift_range,
           shear_range=shear_range,
           zoom_range=zoom_range,
           horizontal_flip=horizontal_flip,
           fill_mode='nearest')
    
    for classe in classes:
        picts = os.listdir(train_path + '/' + classe)
        print(classe)
        
        for pict in picts:
            img = Image.open(train_path + '/' + classe + '/' + pict)
            img = np.array(img)
            img = img.reshape((1,) + img.shape)
            i=1
            
            for batch in datagen.flow(img, batch_size=1, save_to_dir=train_path, save_prefix=classe + '/' + classe, save_format=save_format):
                i += 1
                if i > coeff_creation:
                    break  # otherwise the generator would loop indefinitely

In [ ]:
#data_augmentation(project_path + 'Split_Data/Airliners/Train', coeff_creation = 2)

In [24]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

In [25]:
train_generator = train_datagen.flow_from_directory(project_path + 'Split_Data/Airliners/Train',
                                                    target_size=(224,224),
                                                    color_mode='rgb',
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    shuffle=True)

Found 5600 images belonging to 8 classes.


In [26]:
train_generator.class_indices

{'737': 0,
 '747': 1,
 '757': 2,
 '777': 3,
 'A320': 4,
 'A321': 5,
 'A330': 6,
 'A350': 7}

In [27]:
test_generator = train_datagen.flow_from_directory(project_path + 'Split_Data/Airliners/Test',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 2400 images belonging to 8 classes.


# Transfer learning

In [28]:
# create the base pre-trained model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# add a global spatial average pooling layer
x = base_model.output

'''x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)'''

#x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)

# let's add a fully-connected layer
x = Dense(1024)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Dense(512)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

# dernière couche que sert a prédire la bonne classe
x = Dense(nb_types)(x)
x = keras.layers.BatchNormalization()(x)
predictions = Softmax()(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [29]:
#model = load_model('model_ext_16.h5')

In [30]:
batch_size = 128

step_size_train=train_generator.n//train_generator.batch_size

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                              patience=1, 
                              verbose=1, 
                              factor=0.5, 
                              min_lr=0.00001)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=10,
                   validation_data = test_generator,
                   callbacks=[reduce_lr])

Epoch 1/10
175/175 [==============================] - 1292s 7s/step - loss: 1.2279 - accuracy: 0.6064 - val_loss: 0.7389 - val_accuracy: 0.7275
Epoch 2/10
175/175 [==============================] - 1279s 7s/step - loss: 0.5156 - accuracy: 0.9309 - val_loss: 0.5370 - val_accuracy: 0.7908
Epoch 3/10
175/175 [==============================] - 2799s 16s/step - loss: 0.3189 - accuracy: 0.9768 - val_loss: 0.5975 - val_accuracy: 0.7987
Epoch 4/10
175/175 [==============================] - 2056s 12s/step - loss: 0.2402 - accuracy: 0.9862 - val_loss: 0.5833 - val_accuracy: 0.8208
Epoch 5/10
175/175 [==============================] - 1325s 8s/step - loss: 0.2142 - accuracy: 0.9861 - val_loss: 0.5137 - val_accuracy: 0.8171

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 6/10
175/175 [==============================] - 2396s 14s/step - loss: 0.1605 - accuracy: 0.9925 - val_loss: 0.5471 - val_accuracy: 0.8404
Epoch 7/10
175/175 [==============================] 

In [31]:
# Save model and labels
os.makedirs(project_path + 'Models/' + model_name + '/', exist_ok=True)
save_model_classes(project_path + 'Models/',
                   model_name, train_generator, model)